In [2]:
# Stats on Ca2+ imaging with miniscope and Osc

#######################################################################################
                            # Define Experiment type #
#######################################################################################

DrugExperiment=1 #if CGP Experiment
#DrugExperiment=0 #if Baseline Experiment
suffix='_AH_FINAL'

choosed_folder='Osc_2024_08_04_10_35_33_AH_FINAL' if DrugExperiment else 'Osc_2024_08_04_09_24_52_AH_FINAL'
PrefVigExcel_file = 'AVG_VigSt_2024-08-04_13_06_25_AB_FINAL' if DrugExperiment else 'AVG_VigSt_2024-08-04_10_42_04_AB_FINAL'
                
#######################################################################################
                                # Load packages #
#######################################################################################

import statsmodels.api as sm
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import zscore
from scipy.stats import sem
from datetime import datetime
import shutil
from scipy import stats
from scipy.stats import t
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore")


def column_with_max_single_per_row(row):
    max_val = row.max()  # Find the max value in the row
    max_columns = row == max_val  # Boolean Series of columns with max value
    
    if max_columns.sum() > 1:
        return 'NoPref'  # More than one column has the max value
    else:
        return max_columns.idxmax()  # Return the name of the column with max value

########################################################################
        # SCRIPT 27AB_GrandAverages&Stats_for_Osc
########################################################################

# Specify the directory containing the Excel files
InitialDirectory = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/AB_Analysis" if DrugExperiment else "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis"
directory= f'{InitialDirectory}/{choosed_folder}'

# Get the current date and time
FolderNameSave=str(datetime.now())[:19]
FolderNameSave = FolderNameSave.replace(" ", "_").replace(".", "_").replace(":", "_")
destination_folder= f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/AB_Analysis/AVG_Osc_{FolderNameSave}{suffix}/" if DrugExperiment else f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis/AVG_Osc_{FolderNameSave}{suffix}/"
os.makedirs(destination_folder)
folder_to_save=Path(destination_folder)

# Copy the script file to the destination folder
source_script = "C:/Users/Manip2/SCRIPTS/CodePythonAudrey/CodePythonAurelie/HayLabAnalysis/python/25_28_GrandAverage&Stats_for_DownStatesSpdl&SWR_FullAuto.ipynb"
destination_file_path = f"{destination_folder}/25_28_GrandAverage&Stats_for_DownStatesSpdl&SWR_FullAuto.txt"
shutil.copy(source_script, destination_file_path)

NrSubtypeList=['L2&3', 'L1']
CTX=['PFC_', 'S1_', 'S1PFC_']
Coupling=['', 'UnCoupled', 'Coupled']
OscList=['Spdl', 'SWR']#, 'DS']
Drugs= ['Baseline', 'CGP'] if DrugExperiment else ['Baseline']

for o, Osc in enumerate(OscList): 
    
    print(Osc, 'oscillations analysis...')

    AllOscStatut=pd.DataFrame()
    AllOscDuration=pd.DataFrame()
    AllOscCoupling=pd.DataFrame()
    AllGlobalSpindle=pd.DataFrame()
    AllOscStartLocation=pd.DataFrame()
    
    AllOscStatutS1=pd.DataFrame()
    AllOscDurationS1=pd.DataFrame()
    AllOscCouplingS1=pd.DataFrame()

    AllOscStatutPFC=pd.DataFrame()
    AllOscDurationPFC=pd.DataFrame()
    AllOscCouplingPFC=pd.DataFrame()

    AllOscStatutS1PFC=pd.DataFrame()
    AllOscDurationS1PFC=pd.DataFrame()
    AllOscCouplingS1PFC=pd.DataFrame()

    for NrSubtype in NrSubtypeList: 
        
        print('... for', NrSubtype, 'neurons...')
        
        # Initialize an empty list to store the dataframes
        dfs = []
        df=[]
        dfs2 = []
        df2=[]
        dfs2_per_sheet = {}
        dfs3 = []
        df3=[]
        dfs3_per_sheet = {}
        dfs4 = []
        df4=[]
        dfs4_per_sheet = {}
        filtered_df=[]

        if NrSubtype=='L1':
            MiceList=['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK', 'GreenLinesOK', 'RedLinesOK']
        else:
            MiceList=['Purple', 'ThreeColDotsOK', 'ThreeBlueCrossesOK']

        nametofind=f'{Osc}_Global'
        nametofind2=f'{Osc}_CaPSTH'
        nametofind3=f'{Osc}_SpPSTH'
        #nametofind4=f'{OscillationList[o]}_{Cortex}_SpikeSum'

        # Recursively traverse the directory structure
        for root, _, files in os.walk(directory):
            for filename in files:
                # Check if the file is an Excel file and contains the specified name
                if filename.endswith('.xlsx') and nametofind in filename:
                    if any(name in filename for name in MiceList):  
                        # Construct the full path to the file
                        filepath = os.path.join(root, filename)
                        # Read the Excel file into a dataframe and append it to the list
                        df = pd.read_excel(filepath, index_col=0)
                        dfs.append(df)
                        print(filename)
                if filename.endswith('.xlsx') and nametofind2 in filename: 
                    if any(name in filename for name in MiceList): 
                        # Construct the full path to the file
                        filepath = os.path.join(root, filename)
                        # Read the Excel file into a dataframe and append it to the list
                        excel_data = pd.read_excel(filepath, sheet_name=None, index_col=0, header=None)           
                        for sheet_name, df2 in excel_data.items():
                            if len(df2)>0:
                                if sheet_name in dfs2_per_sheet:                                       
                                    updated_matrix = pd.concat([dfs2_per_sheet[sheet_name], df2], ignore_index=False, axis=0)                    
                                    dfs2_per_sheet[sheet_name] = updated_matrix    
                                else:
                                    dfs2_per_sheet[sheet_name] = df2  #one average trace per unique unit, len(df2)==nb unit recorded for that mouse
                        print(filename)
                if filename.endswith('.xlsx') and nametofind3 in filename: 
                    if any(name in filename for name in MiceList): 
                        # Construct the full path to the file
                        filepath = os.path.join(root, filename)
                        # Read the Excel file into a dataframe and append it to the list
                        excel_data = pd.read_excel(filepath, sheet_name=None, index_col=0, header=None)           
                        for sheet_name, df3 in excel_data.items():
                            if sheet_name in dfs3_per_sheet:   
                                updated_matrix = pd.concat((dfs3_per_sheet[sheet_name],df3), ignore_index=False, axis=0)                
                                dfs3_per_sheet[sheet_name] = updated_matrix                    
                            else:                    
                                dfs3_per_sheet[sheet_name] = df3 #one average trace per unique unit, len(df3)==nb unit recorded for that mouse
                        print(filename)

        ###########################################################################
                                    ##### GLOBAL #####
        ###########################################################################

        # Concatenate all dataframes into a single dataframe
        try: combined_df = pd.concat(dfs, ignore_index=True)
        except: combined_df =pd.DataFrame(dfs)

        combined_df['Unique_Unit'] = combined_df['Unique_Unit'].astype(str)
        combined_df['UnitNumber'] = combined_df['UnitNumber'].astype(str)
        combined_df['UnitValue'] = combined_df['UnitValue'].astype(str)

        combined_df[f'{Osc}Statut'] = combined_df[f'{Osc}Statut'].astype(str)

        combined_df['Unit_ID'] = combined_df['Mice'] + combined_df['Unique_Unit']
        combined_df['Session_ID'] = combined_df['Mice'] + combined_df['Session'].astype(str)

        unique_count = combined_df['Unit_ID'].nunique()
        print(unique_count, f'{NrSubtype} neurons recorded') 

        # Remove non defined Unique Units 
        combined_df = combined_df[combined_df['Unique_Unit'] != '[]']
        combined_df = combined_df.dropna(subset=['Unique_Unit'])
        unique_count = combined_df['Unit_ID'].nunique()
        print(unique_count, f'{NrSubtype} neurons in the cross-registration') 
        
        combined_df[f'{Osc}_ID'] = combined_df['Mice'] + combined_df['Session'] + combined_df[f'{Osc}Number'].astype(str)
        
        unique_count = combined_df[f'{Osc}_ID'].nunique()
        print(unique_count, f'{Osc} recorded in total')

        filenameOut = f'{folder_to_save}/{NrSubtype}_{Osc}_Global.xlsx'
        writer = pd.ExcelWriter(filenameOut)
        combined_df.to_excel(writer)
        writer.close()

        for drug in Drugs: 
            
            combined_df_Drug = combined_df.copy()
            try:
                combined_df_Drug = combined_df_Drug[combined_df_Drug['Drug'] == drug]
            except: 
                combined_df_Drug=combined_df_Drug
            
            folder_to_save2= f'{folder_to_save}/{drug}/'
            if NrSubtype=='L2&3' and o==0 :
                os.makedirs(folder_to_save2)

            #####################
            # PREFERENCE #
            #####################
                        
            # Load the Excel file and read each sheet into a separate DataFrame
            excel_file = f'{InitialDirectory}/{PrefVigExcel_file}/Baseline/{NrSubtype}_ActivityPreference.xlsx'
            sheets = pd.read_excel(excel_file, sheet_name=None, header=None, index_col=0)  # sheet_name=None reads all sheets
            AllUnits=combined_df_Drug['Unit_ID'].unique()
            sheets['AllUnits']=pd.DataFrame(AllUnits)

            # Print the names of the sheets and their corresponding DataFrames
            for List_name, listI in sheets.items():
                
                list=listI[0].tolist() if List_name=='AllUnits' else listI[1].tolist()
                filtered_df = combined_df_Drug[combined_df_Drug['Unit_ID'].isin(list)]
                List_name=List_name.replace( '\\', '/')

                if NrSubtype=='L2&3' and o==0:
                    new_folder= f"{folder_to_save2}/{List_name}/"
                    os.makedirs(new_folder)

                ###########################################################################
                                            ##### PSTH #####
                ###########################################################################

                filenameOutAVG = f'{folder_to_save2}/{List_name}/{NrSubtype}_{Osc}_AVG&SEM.xlsx'
                excel_writerAVG = pd.ExcelWriter(filenameOutAVG)
                filenameOutAVGBSL = f'{folder_to_save2}/{List_name}/{NrSubtype}_{Osc}_BSL_AVG&SEM.xlsx'
                excel_writerAVGBSL = pd.ExcelWriter(filenameOutAVGBSL)

                filenameOutMean = f'{folder_to_save2}/{List_name}/{NrSubtype}_{Osc}_During.xlsx'
                excel_writerMean= pd.ExcelWriter(filenameOutMean)
                filenameOutMeanBSL = f'{folder_to_save2}/{List_name}/{NrSubtype}_{Osc}_BSL_During.xlsx'
                excel_writerMeanBSL= pd.ExcelWriter(filenameOutMeanBSL)

                FileNames= ['CaPSTH', 'SpPSTH'] # 'SpikeGrandSum',
                FileNamesNormalized= ['BSL_CaPSTH', 'BSL_SpPSTH'] #, 'Baselined_SpikeGrandSum',
                dfs_per_sheet=[dfs2_per_sheet,dfs3_per_sheet]# dfs4_per_sheet, ] #Ca &Sp

                for d, df_per_sheet in enumerate(dfs_per_sheet):

                    FileName=FileNames[d]
                    FileNameNormalized=FileNamesNormalized[d]
                    BigArray=pd.DataFrame()
                    BigArrayBSL=pd.DataFrame()
                    AVGArray=pd.DataFrame()
                    AVGArrayBSL=pd.DataFrame()

                    filenameOut = f'{folder_to_save2}/{List_name}/{NrSubtype}_{Osc}_{FileName}.xlsx'
                    excel_writer = pd.ExcelWriter(filenameOut)
                    
                    filenameOutBSL = f'{folder_to_save2}/{List_name}/{NrSubtype}_{Osc}_{FileNameNormalized}.xlsx'
                    excel_writerBSL = pd.ExcelWriter(filenameOutBSL)
                        
                    for ctx in CTX:
                        for coup in Coupling: 

                            ctx= '' if Osc=='SWR' else ctx

                            Array=[]
                            Array=pd.DataFrame(df_per_sheet[f'{ctx}{coup}{Osc}_{drug}'])
                        
                            # Only keep the neurons belonging to the list (All units, NREM active, REM active, etc)
                            present_indices = [idx for idx in list if idx in Array.index]
                            Array = Array.loc[present_indices] 
                        
                            # Leave a blanck space for units not recorded in that condition
                            missing_indexes = set(list) - set(Array.index)
                            Array = Array.reindex(Array.index.union(missing_indexes))
                            Array = Array.sort_index()
                            Array.to_excel(excel_writer, sheet_name=f'{ctx}{coup}{Osc}', index=True, header=False)

                            mArray=Array.mean(axis=0)
                            semArray = stats.sem(Array, axis=0, ddof=1, nan_policy='omit')
                            icArray = norm.ppf((1 +  0.95) / 2) * (np.std(Array, axis=0) / np.sqrt(Array.shape[0]))
                            SmallArray=pd.DataFrame(np.transpose([mArray,semArray,icArray]), columns=[f'{ctx}{coup}{Osc} Mean', f'{ctx}{coup}{Osc} SEM', f'{ctx}{coup}{Osc} IC'])
                            BigArray=pd.concat([BigArray,SmallArray], axis=1)

                            SecondHalf_columns = Array.iloc[:, Array.shape[1]//2:]
                            mean_baseline = SecondHalf_columns.mean(axis=1)
                            avgarray=pd.DataFrame(mean_baseline, columns=[f'{ctx}{coup}{Osc}'])
                            AVGArray=pd.concat([AVGArray,avgarray], axis=1)
                            
                            # Baseline signals
                            baseline_columns = Array.iloc[:, :Array.shape[1] // 4]
                            mean_baseline = baseline_columns.mean(axis=1)
                            Array = Array.sub(mean_baseline, axis=0)
                            Array.to_excel(excel_writerBSL, sheet_name=f'{ctx}{coup}{Osc}', index=True, header=False)

                            mArray=Array.mean(axis=0)
                            semArray = stats.sem(Array, axis=0, ddof=1, nan_policy='omit')
                            icArray = norm.ppf((1 +  0.95) / 2) * (np.std(Array, axis=0) / np.sqrt(Array.shape[0]))
                            SmallArray=pd.DataFrame(np.transpose([mArray,semArray,icArray]), columns=[f'{ctx}{coup}{Osc} Mean', f'{ctx}{coup}{Osc} SEM', f'{ctx}{coup}{Osc} IC'])
                            BigArrayBSL=pd.concat([BigArrayBSL,SmallArray], axis=1)

                            SecondHalf_columns = Array.iloc[:, Array.shape[1]//2:]
                            mean_baseline = SecondHalf_columns.mean(axis=1)
                            avgarray=pd.DataFrame(mean_baseline, columns=[f'{ctx}{coup}{Osc}'])
                            AVGArrayBSL=pd.concat([AVGArrayBSL,avgarray], axis=1)

                    excel_writer.close()
                    excel_writerBSL.close() 
                    BigArray.to_excel(excel_writerAVG, sheet_name=FileName, index=True, header=True)
                    AVGArray.to_excel(excel_writerMean, sheet_name=FileName, index=True, header=True)
                    BigArrayBSL.to_excel(excel_writerAVGBSL, sheet_name=FileNameNormalized, index=True, header=True)
                    AVGArrayBSL.to_excel(excel_writerMeanBSL, sheet_name=FileNameNormalized, index=True, header=True)

                excel_writerAVG.close()                   
                excel_writerAVGBSL.close()    
                excel_writerMean.close()                   
                excel_writerMeanBSL.close()    

        #######################
        # Propreties Osc
        #######################
        filenameOut = f'{folder_to_save}/{Osc}Propreties.xlsx'
        writer = pd.ExcelWriter(filenameOut)

        combined_dfOsc = combined_df.drop_duplicates(subset=f'{Osc}_ID', keep='first')
        #All Spdl
        OscStatut = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}Statut']])
        AllOscStatut=pd.concat([AllOscStatut, OscStatut], axis=0)
        OscDuration = combined_dfOsc.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
        AllOscDuration=pd.concat([AllOscDuration, OscDuration], axis=0)
        OscCoupling = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'SWR_inside_Spdl']])
        AllOscCoupling=pd.concat([AllOscCoupling, OscCoupling], axis=0)

        if Osc== 'Spdl':
            GlobalSpindle = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'GlobalSpindle']])
            AllGlobalSpindle=pd.concat([AllGlobalSpindle, GlobalSpindle], axis=0)
            OscStartLocation = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}StartLocation']])
            AllOscStartLocation=pd.concat([AllOscStartLocation, OscStartLocation], axis=0)

            #S1 Spdl
            combined_dfOscS1=combined_dfOsc[combined_dfOsc[f'{Osc}StartLocation']=='S1' and combined_dfOscS1[f'GlobalSpindle']=='Local']
            OscStatut = pd.crosstab(index=combined_dfOscS1['Session_ID'],columns=[combined_dfOscS1['Drug'], combined_dfOscS1[f'{Osc}Statut']])
            AllOscStatutS1=pd.concat([AllOscStatut, OscStatut], axis=0)
            OscDuration = combined_dfOscS1.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
            AllOscDurationS1=pd.concat([AllOscDuration, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOscS1['Session_ID'],columns=[combined_dfOscS1['Drug'], combined_dfOscS1[f'SWR_inside_Spdl']])
            AllOscCouplingS1=pd.concat([AllOscCoupling, OscCoupling], axis=0)

            #PFC Spdl
            combined_dfOscPFC=combined_dfOsc[combined_dfOsc[f'{Osc}StartLocation']=='PFC' and combined_dfOscS1[f'GlobalSpindle']=='Local']
            OscStatut = pd.crosstab(index=combined_dfOscPFC['Session_ID'],columns=[combined_dfOscPFC['Drug'], combined_dfOscPFC[f'{Osc}Statut']])
            AllOscStatutPFC=pd.concat([AllOscStatut, OscStatut], axis=0)
            OscDuration = combined_dfOscPFC.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
            AllOscDurationPFC=pd.concat([AllOscDuration, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOscPFC['Session_ID'],columns=[combined_dfOscPFC['Drug'], combined_dfOscPFC[f'SWR_inside_Spdl']])
            AllOscCouplingPFC=pd.concat([AllOscCoupling, OscCoupling], axis=0)

            #S1PFC Spdl
            combined_dfOscS1PFC=combined_dfOsc[combined_dfOsc[combined_dfOscS1[f'GlobalSpindle']=='Global']]
            OscStatut = pd.crosstab(index=combined_dfOscS1PFC['Session_ID'],columns=[combined_dfOscS1PFC['Drug'], combined_dfOscS1PFC[f'{Osc}Statut']])
            AllOscStatutS1PFC=pd.concat([AllOscStatut, OscStatut], axis=0)
            OscDuration = combined_dfOscS1PFC.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
            AllOscDurationS1PFC=pd.concat([AllOscDuration, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOscS1PFC['Session_ID'],columns=[combined_dfOscS1PFC['Drug'], combined_dfOscS1PFC[f'SWR_inside_Spdl']])
            AllOscCouplingS1PFC=pd.concat([AllOscCoupling, OscCoupling], axis=0)


    AllOscStatut.to_excel(writer, sheet_name=f'CouplingStatut')
    AllOscDuration.to_excel(writer, sheet_name=f'MeanDuration')
    AllOscCoupling.to_excel(writer, sheet_name=f'SWR_inside_Spdl')    
    if Osc== 'Spdl':
        AllGlobalSpindle.to_excel(writer, sheet_name=f'GlobalSpindle')
        AllOscStartLocation.to_excel(writer, sheet_name=f'StartLocation')

        AllOscStatutS1.to_excel(writer, sheet_name=f'S1_CouplingStatut')
        AllOscDurationS1.to_excel(writer, sheet_name=f'S1_MeanDuration')
        AllOscCouplingS1.to_excel(writer, sheet_name=f'S1_SWR_inside_Spdl') 
        
        AllOscStatutPFC.to_excel(writer, sheet_name=f'PFC_CouplingStatut')
        AllOscDurationPFC.to_excel(writer, sheet_name=f'PFC_MeanDuration')
        AllOscCouplingPFC.to_excel(writer, sheet_name=f'PFC_SWR_inside_Spdl') 
        
        AllOscStatutS1PFC.to_excel(writer, sheet_name=f'S1PFC_CouplingStatut')
        AllOscDurationS1PFC.to_excel(writer, sheet_name=f'S1PFC_MeanDuration')
        AllOscCouplingS1PFC.to_excel(writer, sheet_name=f'S1PFC_SWR_inside_Spdl')

    writer.close()


Spdl oscillations analysis...
... for L2&3 neurons...
Spdl_CaPSTH_Purple.xlsx
Spdl_CaPSTH_ThreeColDotsOK.xlsx
Spdl_Global_Purple.xlsx
Spdl_Global_ThreeColDotsOK.xlsx
Spdl_SpPSTH_Purple.xlsx
Spdl_SpPSTH_ThreeColDotsOK.xlsx
507 L2&3 neurons recorded
507 L2&3 neurons in the cross-registration
775 Spdl recorded in total
... for L1 neurons...
Spdl_CaPSTH_BlackLinesOK.xlsx
Spdl_CaPSTH_BlueLinesOK.xlsx
Spdl_CaPSTH_GreenDotsOK.xlsx
Spdl_Global_BlackLinesOK.xlsx
Spdl_Global_BlueLinesOK.xlsx
Spdl_Global_GreenDotsOK.xlsx
Spdl_SpPSTH_BlackLinesOK.xlsx
Spdl_SpPSTH_BlueLinesOK.xlsx
Spdl_SpPSTH_GreenDotsOK.xlsx
63 L1 neurons recorded
62 L1 neurons in the cross-registration
892 Spdl recorded in total
SWR oscillations analysis...
... for L2&3 neurons...
SWR_CaPSTH_Purple.xlsx
SWR_CaPSTH_ThreeColDotsOK.xlsx
SWR_Global_Purple.xlsx
SWR_Global_ThreeColDotsOK.xlsx
SWR_SpPSTH_Purple.xlsx
SWR_SpPSTH_ThreeColDotsOK.xlsx
507 L2&3 neurons recorded
507 L2&3 neurons in the cross-registration
2113 SWR recorded in t